In [1]:
import numpy as np
import pandas as pd
from city_to_state import city_to_state_dict

In [2]:
data = pd.read_csv('Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.csv')

# Create dictionary of mispelled country names to recognize and set each associated Country correctly
global country_names
country_names={'United States':'USA', 'US':'USA', 'United States of America':'USA', 'United States ':'USA', 'UK ':'UK', 'United Kingdom':'UK', 
               'U.S>':'USA', 'USA ':'USA', 'Usa':'USA', 'usa':'USA', 'Uk':'UK', 'USA-- Virgin Islands':'USA', 'Us':'USA', 'US ':'USA', 
               'U.S.A.':'USA', 'U.S.A':'USA', 'U.S. ':'USA', 'United States':'USA', 'United State':'USA', 'United Kingdom ':'UK', 'Scotland ':'UK',
               'England':'UK', 'England/UK':'UK', 'Scotland':'UK', 'England ':'UK', 'uk':'UK', 'Unites States':'USA', 'Uk ':'UK', 
               'UK (Northern Ireland)':'UK', 'England, UK':'UK', 'U.S.':'USA', 'United Kingdom.':'UK', 'United kingdom':'UK', 'U.K. ':'UK',
               'United kingdom ':'UK', 'England, UK.':'UK', 'United Kingdom (England)':'UK', 'Unted States':'USA', 'ENGLAND':'USA', 'Great Britain':'UK',
               'Us ':'USA', 'UXZ':'USA', 'U.K':'UK', 'United states':'USA', 'united states':'USA', 'United States of America ':'USA', 'Wales, UK':'UK',
               'Northern Ireland, United Kingdom':'UK', 'England, United Kingdom ':'UK', 'England, United Kingdom':'UK', 'America':'USA', 'england':'UK',
               'united stated':'USA', 'Wales (United Kingdom)':'UK', 'Us ':'USA', 'UK (England)':'UK', 'uk':'UK', 'Uk ':'UK', 'England, UK':'UK',
               'U.K. (northern England)':'UK', 'us':'USA', 'U.K.':'UK', 'United Kingdomk':'UK', 'Englang':'UK', 'Wales (UK)':'UK', 'Scotland, UK':'UK',
               'United States- Puerto Rico':'USA', 'UK, remote':'UK', 'UK for U.S. company':'USA', 'Canada ': 'Canada', 'canada':'Canada', 'CANADA':'Canada',
               'Canadw':'Canada', 'Can':'Canada', 'Canda':'Canada', 'Puerto Rico ':'USA', 'England, Gb':'UK', 'UK for U.S. company':'USA', 'Canadá':'Canada',
               'United Kindom':'UK', 'ireland':'Ireland', 'Ireland ':'Ireland', 'Japan ':'Japan', 'Sweden ':'Sweden', 'United Arab Emirates ':'UAE', 
               'Unites kingdom ':'UK', 'Northern Ireland ':'UK', 'united kingdom':'UK', 'Wales':'UK', 'Canada, Ottawa, ontario':'Canada', 'Britain ':'UK',
               'FRANCE':'France', 'CANADA ':'Canada', 'spain':'Spain', 'Canad':'Canada', 'Australia ':'Australia', 'australia':'Australia', 
               'Australi':'Australia', 'Csnada':'Canada', 'France ':'France', 'Puerto Rico':'USA', 'Northern Ireland':'UK', 'germany':'Germany',
               'New Zealand Aotearoa':'New Zealand', 'Aotearoa New Zealand':'New Zealand', 'france':'France', 'New zealand':'New Zealand',
               'New Zealand ':'New Zealand', ' New Zealand':'New Zealand', 'Germany ':'Germany', 'Mexico ':'Mexico', 'Great Britain ': 'UK',
               'finland':'Finland', 'Russia ':'Russia', 'NZ':'New Zealand', 'The Netherlands':'Netherlands', 'Netherlands ':'Netherlands',
               'the netherlands':'Netherlands', 'new zealand':'New Zealand', 'INDIA':'India', 'Spain ':'Spain', 'The netherlands': 'Netherlands', 
               'South Korea ':'South Korea', 'Ukraine ':'Ukraine', 'hong konh':'Hong Kong', 'Hong Kong ':'Hong Kong', 'switzerland':'Switzerland',
               'Brazil ':'Brazil', 'SWITZERLAND':'Switzerland', 'japan':'Japan', 'Australian ':'Australia', 'The Netherlands ':'Netherlands',
               'netherlands':'Netherlands', 'South Africa ':'South Africa', 'Croatia ':'Croatia', 'Czech republic':'Czech Republic', 
               'czech republic':'Czech Republic', 'Czech Republic ':'Czech Republic', 'Italy (South)':'Italy', 'Bangladesh ':'Bangladesh', 
               'singapore':'Singapore', 'Malaysia ':'Malaysia', 'Singapore ':'Singapore', 'Denmark ':'Denmark', 'denmark':'Denmark', 'Norway ':'Norway',
               'Japan, US Gov position':'USA','Remote (philippines)':'USA', 'croatia':'Croatia', 'Company in Germany. I work from Pakistan.':'Germany', 
               'ARGENTINA BUT MY ORG IS IN THAILAND':'Thailand', 'Czechia':'Czech Republic', 'Jersey, Channel islands':'UK', 'Mainland China':'China',
               'Switzerland ': 'Switzerland', 'South africa':'South Africa', 'México':'Mexico', 'Catalonia':'Spain', 'Panamá':'Panama', 'the Netherlands':'Netherlands',
               'Nederland':'Netherlands', 'Brasil':'Brazil', 'Portugal ':'Portugal', 'Sri lanka':'Sri Lanka', 'The Bahamas ':'The Bahamas', 'NL':'Netherlands',
               'europe':'Czech Republic', 'Belgium ':'Belgium'}

In [5]:
def clean_columns(df):
    df = df.drop('Additional context on job title',axis=1)
    df = df.drop('Additional context on income',axis=1)


    #Condense experience down to only 1 column
    k = df['Overall years of professional experience'].unique()
    exp = {}
    for carrot in k:
        exp[carrot] = 0
    exp['2 - 4 years'] = 2
    exp['5-7 years'] = 5
    exp['8 - 10 years'] = 8
    exp['11 - 20 years'] = 11
    exp['21 - 30 years'] = 21
    exp['31 - 40 years'] = 31
    exp['41 years or more'] = 41
    df['Overall years of professional experience'] = df['Overall years of professional experience'].apply(lambda x: exp.get(x))
    df['Years of experience in field'] = df['Years of experience in field'].apply(lambda x: exp.get(x))
    
    # Correct years of experience if field experience is greater than professional experience and condense both columns into one
    df['Experience'] = df['Overall years of professional experience']
    df.loc[df['Overall years of professional experience'] < df['Years of experience in field'], 'Experience'] = df['Years of experience in field']    
    df = df.drop(['Overall years of professional experience'], axis=1)
    df = df.drop(['Years of experience in field'], axis=1)
    
    # Create list of states to recognize and set each associated Country as 'USA'
    statesL = ['alabama', 'alaska', 'arizona', 'arkansas', 'california', 'colorado', 'connecticut', 'delaware', 'florida', 'georgia', 'hawaii', 'idaho', 'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine', 'maryland', 'massachusetts', 'michigan', 'minnesota', 'mississippi', 'missouri', 'montana', 'nebraska', 'nevada', 'new hampshire', 'new jersey', 'new mexico', 'new york', 'north carolina', 'north dakota', 'ohio', 'oklahoma', 'oregon', 'pennsylvania', 'rhode island', 'south carolina', 'south dakota', 'tennessee', 'texas', 'utah', 'vermont', 'virginia', 'washington', 'west virginia', 'wisconsin', 'wyoming', 'puerto rico', 'district of columbia']
    states=[]
    for i in statesL:
        new = i.title()
        if new == 'District Of Columbia':
            new = 'District of Columbia'
        states.append(new)
    df.loc[((df.State.isin(states)) | (df.City.isin(states)) | (df.State.isin(city_to_state_dict.keys()))), 'Country'] = 'USA'
    
    df['Country'].replace(country_names, inplace=True)
    country1=df['Country'].value_counts().tail(49).keys().tolist()
    df=df[~df['Country'].isin(country1)]
    return df

new_data = clean_columns(data)
new_data.head(10)

,Timestamp,How old are you?,Industry,Job title,Annual salary,Other monetary comp,Currency,Currency - other,Country,State,City,Highest level of education completed,Gender,Race,Experience
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,"55,000",0.0,USD,NaN,USA,Massachusetts,Boston,Master's degree,Woman,White,5
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,"54,600",4000.0,GBP,NaN,UK,NaN,Cambridge,College degree,Non-binary,White,8
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,"34,000",NaN,USD,NaN,USA,Tennessee,Chattanooga,College degree,Woman,White,2
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,"62,000",3000.0,USD,NaN,USA,Wisconsin,Milwaukee,College degree,Woman,White,8
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,"60,000",7000.0,USD,NaN,USA,South Carolina,Greenville,College degree,Woman,White,8
5,4/27/2021 11:02:46,25-34,Education (Higher Education),Scholarly Publishing Librarian,"62,000",NaN,USD,NaN,USA,New Hampshire,Hanover,Master's degree,Man,White,8
6,4/27/2021 11:02:51,25-34,Publishing,Publishing Assistant,"33,000",2000.0,USD,NaN,USA,South Carolina,Columbia,College degree,Woman,White,2
7,4/27/2021 11:03:00,25-34,Education (Primary/Secondary),Librarian,"50,000",NaN,USD,NaN,USA,Arizona,Yuma,Master's degree,Man,White,5
8,4/27/2021 11:03:01,45-54,Computing or Tech,Systems Analyst,"112,000",10000.0,USD,NaN,USA,Missouri,St. Louis,College degree,Woman,White,21
9,4/27/2021 11:03:02,35-44,"Accounting, Banking & Finance",Senior Accountant,"45,000",0.0,USD,NaN,USA,Florida,Palm Coast,College degree,Woman,"Hispanic, Latino, or Spanish origin, White",21


In [6]:

# df['Country'].replace(country_names, inplace=True)
# df['Country'].unique()

# USA                22948
# Canada              1664
# UK                  1572
# Australia            383
# Germany              171
# Ireland              124
# New Zealand          122
# Global, Remote, International, Africa, europe, Policy

# print(len(df['Country'].unique()))
# country1=df['Country'].value_counts().tail(49).keys().tolist()
# print(country1)
# print(df['Country'].unique())
# list_once=country1.keys().tolist()
# df=df[~df['Country'].isin(list_once)]
# print(len(df['Country'].unique()))